# CIFAR 10 Image Classification

Most of the code cells below include comments explaining the task to be performed in those cells. Please delete the comments and add code to perform those tasks. There are a few code cells in which code has already been provided for you. In some cases, you will need to complete this code.

⚠ **NOTE:** You should make use of GPU acceleration in this notebook. 



# Import Packages

In [ ]:
# Import numpy, pandas, and matplotlib using the standard aliases. 
# Import mpimg from matplotlib.image
# Import train_test_split from sklearn
# Import pickle. 
# Import tensorflow and all needed tools from tensorflow.keras. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split


import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load Training DataFrame

In [ ]:
# Load the training data into a DataFrame named 'train'. 
# Print the shape of the resulting DataFrame. 
# You do not need the test data in this notebook. 
train = pd.read_csv(f'../input/mu-cifar10/train.csv')

print('Training Set Size:', train.shape)

In [ ]:
# Display the head of the train DataFrame. 
train.head()

In [ ]:
# Convert 'label' to string
train['label'] = train['label'].astype(str)

# Label Distribution

In [ ]:
# Display a DataFrame showing the proportion of observations with each 
# possible of the target variable (which is label). 
(train.label.value_counts() / len(train)).to_frame()

# View Sample of Images

In [ ]:
# Sample 16 images from the training set and display these along with their labels.
# The images should be arranged in a 4x4 grid of subplots. 
# Please set the figure sizeto (6,6)

sample = train.sample(n=16).reset_index()

plt.figure(figsize=(6,6))

for i, row in sample.iterrows():

    img = mpimg.imread(f'../input/mu-cifar10/train_images/{row.filename}')    
    label = row.label

    plt.subplot(4,4,i+1)
    plt.imshow(img)
    plt.text(0, -5, f'Class {label}', color='k')
        
    plt.axis('off')

plt.tight_layout()
plt.show()

# Data Generators

In [ ]:
# Split the dataframe train into two DataFrames named train_df and valid_df. 
# Use 20% of the data for the validation set. 
# Use stratified sampling so that the label proportions are preserved.
# Set a random seed for the split. 

train_df, valid_df = train_test_split(train, test_size=0.2, random_state=1, stratify=train.label)

print(train_df.shape)
print(valid_df.shape)

In [ ]:
# Create image data generators for both the training set and the validation set. 
# Use the data generators to scale the pixel values by a factor of 1/255. 

train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train.label = train.label.astype(str)

In [ ]:
# Complete the code for the data loaders below. 

BATCH_SIZE = 64

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = f'../input/mu-cifar10/train_images' ,
    x_col = 'filename',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

valid_loader = train_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = f'../input/mu-cifar10/train_images',
    x_col = 'filename',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = False,   # no need to shuffle validation set 
    class_mode = 'categorical',
    target_size = (32,32)
)

In [ ]:
# Run this cell to determine the number of training and validation batches. 

TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

# Build Network

In [ ]:
# Use this cell to construct a convolutional neural network model. 
# Your model should make use of each of the following layer types:
#    Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense
# You can start by mimicking the architecture used in the 
# Aerial Cactus competetition, but you should explore different architectures
# by adding more layers and/or adding more nodes in individual layers

np.random.seed(1)
tf.random.set_seed(1)

cnn = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(10, activation='softmax')   #10 values for the 'label'
])

cnn.summary()

# Train Network

# Training Run 1 

In [ ]:
# Define an optimizer and select a learning rate. 
# Then compile the model. 
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

 Complete one or more training runs. 
 Display training curves after each run.

In [ ]:
 %%time 

h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 20,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
history = h1.history
print(history.keys())

In [ ]:
epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

# Training Run 2

In [ ]:
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

In [ ]:
%%time 

h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
for k in history.keys():
    history[k] += h2.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

# Save Model and History

In [ ]:
# When you are satisfied with the model you have found, 
# save the model and the combined history dictionary to files.
# Download these filesto your local device and then upload them 
# as a Kaggle dataset. 

cnn.save('cifarmodel2_v01.h5')
pickle.dump(history, open(f'cifarhistory2_v01.pkl', 'wb'))